In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import dqn
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.qvalue_head = nn.Linear(features_dim, num_outputs)
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.qvalue_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        qvalues = self.qvalue_head(features)
        return qvalues, state

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        print(rew)
        if rew == 0:
            rew = -0.02
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=500)
    env.update_taskset(TaskSet(preset=['C32']))
    env = PovOnlyWrapper(env)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    #env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.dqn import ApexTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(ApexTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "gamma": 0.95,
             "num_gpus": 1,
             "num_workers": 6,
             "buffer_size": 5000000,
             "learning_starts": 5000,
             "train_batch_size": 5000,
             "target_network_update_freq": 5000,
             "prioritized_replay_alpha": 0.5,
             "final_prioritized_replay_beta": 1.0,
             "min_iter_time_s": 10,
             "rollout_fragment_length": 8,
             "collect_metrics_timeout": 1800,
             "exploration_config": {
                  "initial_epsilon": 1,
                  "epsilon_timesteps": 500000,
                  "final_epsilon": 0.0,
              },
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "APEX C32 pretrained (AnnaCNN)"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
APEX_my_env_a0624_00000,PENDING,


2021-10-11 11:44:43,840	INFO wandb.py:170 -- Already logged into W&B.
2021-10-11 11:44:43,900	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=23009) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23009)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=23009) 2021-10-11 11:44:50,893	INFO dqn.py:188 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=23009) 2021-10-11 11:44:50,893	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=23017) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23017)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23007) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23007)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23011) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32

(pid=23010) 2021-10-11 11:47:25,539	WARNING replay_buffer.py:44 -- Estimated max memory usage for replay buffer is 30.78625 GB (1250000.0 batches of size 1, 24629 bytes each), available system memory is 118.205505536 GB
(pid=23003) 2021-10-11 11:47:32,818	WARNING replay_buffer.py:44 -- Estimated max memory usage for replay buffer is 30.78625 GB (1250000.0 batches of size 1, 24629 bytes each), available system memory is 118.205505536 GB
(pid=23012) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23012)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23014) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23014)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23013) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning:

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_my_env_a0624_00000,RUNNING,192.168.1.96:23009,3,2650.53,75040,7.32,10,0,130.23


(pid=23007) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23007)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23014) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23014)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23012) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23012)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23011) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23011)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23013) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/lo

Result for APEX_my_env_a0624_00000:
  agent_timesteps_total: 100072
  custom_metrics: {}
  date: 2021-10-11_12-42-00
  done: false
  episode_len_mean: 132.01
  episode_media: {}
  episode_reward_max: 10.0
  episode_reward_mean: 7.43
  episode_reward_min: 0.0
  episodes_this_iter: 55
  episodes_total: 234
  experiment_id: 243561e26b0c41869a55ea50c7801cd6
  hostname: cdsserver
  info:
    exploration_infos:
    - cur_epsilon: 0.0
      last_timestep: 0
    - cur_epsilon: 0.4
      last_timestep: 99711
    - cur_epsilon: 0.1109031749048234
      last_timestep: 99031
    - cur_epsilon: 0.030748785509924638
      last_timestep: 98271
    - cur_epsilon: 0.008525344843795182
      last_timestep: 98271
    - cur_epsilon: 0.0023637195258383783
      last_timestep: 99983
    - cur_epsilon: 0.0006553600000000003
      last_timestep: 98311
    last_target_update_ts: 18190000
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 0.0877395048737

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_my_env_a0624_00000,RUNNING,192.168.1.96:23009,4,3414.33,100072,7.43,10,0,132.01


(pid=23013) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23013)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23014) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23014)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23007) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23007)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=23017) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=23017)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
